In [ ]:
!pip install scikit-learn==1.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 85.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.4.0
    Uninstalling scikit-learn-1.4.0:
      Successfully uninstalled scikit-learn-1.4.0


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import joblib
# Import the train_test_split function
from sklearn.model_selection import train_test_split

# Télécharger les stopwords si ce n'est pas encore fait
nltk.download('stopwords')

# Charger le dataset
df = pd.read_csv("spam.csv", encoding="latin-1")[["v1", "v2"]]
df.columns = ["label", "message"]

# Convertir les labels en valeurs binaires (spam=1, non spam=0)
df["label"] = df["label"].map({"ham": 0, "spam": 1})

# Nettoyage du texte
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    words = text.split()
    words = [word for word in words if word not in stopwords.words('english')]
    return " ".join(words)

df["cleaned_message"] = df["message"].apply(clean_text)

# Séparation des données
X_train, X_test, y_train, y_test = train_test_split(df["cleaned_message"], df["label"], test_size=0.2, random_state=42)

# Vectorisation
vectorizer = TfidfVectorizer(max_features=10000)
X_train_vec = vectorizer.fit_transform(X_train)  # Assurez-vous d'utiliser fit_transform ici
X_test_vec = vectorizer.transform(X_test)

# Sur-échantillonnage de la classe minoritaire
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_vec, y_train)

# Entraînement du modèle
model = LogisticRegression()
model.fit(X_train_res, y_train_res)

# Sauvegarde du vectorizer et du modèle
joblib.dump(vectorizer, 'tfidf_vectorizer.joblib')  # Sauvegarde du vectorizer ajusté
joblib.dump(model, 'spam_classifier.joblib')  # Sauvegarde du modèle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['spam_classifier.joblib']